In [25]:
import pandas as pd
import datetime as dt
import pymongo
import pprint
from sqlalchemy import create_engine

### Extract CSVs into DataFrames

In [2]:
HuffPost_csv = "HuffPost_trump_approval.csv"


In [3]:
FiveThirtyEight_csv = "FiveThirtyEight_president_approval_polls.csv"


### Transform HuffPost DataFrame

In [4]:
HuffPost = pd.read_csv(HuffPost_csv)

HuffPost.head()

,survey_organization,party_affiliation,start_date,end_date,survey_method,survey_question,survey_population,survey_sample,approve_percent,disapprove_percent,undecided_percent,margin_of_error
0,Gallup,None,2017-01-20,2017-01-22,Live Phone,NaN,Adults,1500.0,45,45,NaN,3.0
1,Ipsos/Reuters,None,2017-01-20,2017-01-24,Internet,"Overall, do you approve or disapprove about th...",Adults,1633.0,43,45,13.0,2.8
2,Ipsos/Reuters,None,2017-01-20,2017-01-24,Internet,"Overall, do you approve or disapprove about th...",Adults (Democrats),680.0,15,75,10.0,4.3
3,Ipsos/Reuters,None,2017-01-20,2017-01-24,Internet,"Overall, do you approve or disapprove about th...",Adults (Republicans),557.0,85,11,4.0,4.7
4,Ipsos/Reuters,None,2017-01-20,2017-01-24,Internet,"Overall, do you approve or disapprove about th...",Adults (Independents),228.0,38,39,23.0,7.4


### Transform FiveThirtyEight DataFrame

In [5]:
FiveThirtyEight = pd.read_csv(FiveThirtyEight_csv)
FiveThirtyEight.head()
# Rename ID column for Mongo load
# counties.rename(columns = { "ID" : "_id"} , inplace = True)
# counties.head()

,question_id,poll_id,cycle,state,politician,pollster_id,pollster,sponsor_ids,sponsors,display_name,...,start_date,end_date,sponsor_candidate,tracking,created_at,notes,url,source,yes,no
0,119718,64736,2020.0,NaN,Donald Trump,568,YouGov,NaN,NaN,YouGov,...,3/4/20,3/6/20,NaN,NaN,3/6/20 11:35,NaN,https://docs.cdn.yougov.com/dojx4dgn4g/tabs_Tr...,538,41.0,51.0
1,119719,64736,2020.0,NaN,Donald Trump,568,YouGov,NaN,NaN,YouGov,...,3/4/20,3/6/20,NaN,NaN,3/6/20 11:35,NaN,https://docs.cdn.yougov.com/dojx4dgn4g/tabs_Tr...,538,44.0,55.0
2,119705,64731,2020.0,NaN,Donald Trump,568,YouGov,NaN,NaN,YouGov,...,3/3/20,3/5/20,NaN,NaN,3/5/20 15:54,NaN,https://docs.cdn.yougov.com/5e41wsj1pi/tabs_Tr...,538,43.0,50.0
3,119706,64731,2020.0,NaN,Donald Trump,568,YouGov,NaN,NaN,YouGov,...,3/3/20,3/5/20,NaN,NaN,3/5/20 15:54,NaN,https://docs.cdn.yougov.com/5e41wsj1pi/tabs_Tr...,538,45.0,53.0
4,119717,64735,2020.0,NaN,Donald Trump,399,Rasmussen Reports/Pulse Opinion Research,NaN,NaN,Rasmussen Reports/Pulse Opinion Research,...,3/3/20,3/5/20,NaN,True,3/6/20 09:28,NaN,http://www.rasmussenreports.com/public_content...,538,47.0,51.0


### Reorganize Columns for Homogeneity

In [6]:
sorted(list(HuffPost["survey_organization"].unique()))

['ARG',
 'CBS',
 'CNN',
 'FOX',
 'Gallup',
 'HuffPost/YouGov',
 'IBD/TIPP',
 'Ipsos/Reuters',
 'McClatchy/Marist',
 'NBC/SurveyMonkey',
 'NBC/WSJ',
 'PPP (D)',
 'Pew',
 'Politico/Morning Consult',
 'Quinnipiac',
 'Rasmussen',
 'SurveyMonkey',
 'YouGov/Economist',
 'Zogby']

In [7]:
sorted(list(FiveThirtyEight["pollster"].unique()))

['ABC News/Washington Post',
 'AP-NORC',
 'America First Policies',
 'American Research Group',
 'AtlasIntel',
 'Basswood Research',
 'CBS News/New York Times',
 'CNN/Opinion Research Corp.',
 'Cards Against Humanity/Survey Sampling International',
 'Civiqs',
 'Civis Analytics',
 'Climate Nexus',
 'Cygnal',
 'D-CYFOR',
 'Emerson College',
 'Fox News/Beacon Research/Shaw & Co. Research',
 'GBAO',
 'GQR Research (GQRR)',
 'Gallup',
 'Garin-Hart-Yang Research Group',
 'Garin-Hart-Yang/Global Strategy Group',
 'Georgetown University (Battleground)',
 'Global Strategy Group/GBA Strategies',
 'Gravis Marketing',
 'Harris Insights & Analytics',
 'Heart+Mind Strategies',
 'IBD/TIPP',
 'Ipsos',
 'Kaiser Family Foundation',
 'Lucid',
 'Marist College',
 'Marquette University Law School',
 'McLaughlin & Associates',
 'Monmouth University',
 'Morning Consult',
 'NBC News/Wall Street Journal',
 'Opinion Savvy/InsiderAdvantage',
 'PSB Research',
 'Pew Research Center',
 'Public Opinion Strategies',


In [8]:
# Create a Huff Post to FiveThirtyEight naming dictionary for overlapping organizations
huff_538_name_dict = {
    "ARG": "American Research Group",
    "CNN": "CNN/Opinion Research Corp.",
    "FOX": "Fox News/Beacon Research/Shaw & Co. Research",
    "Gallup": "Gallup",
    "IBD/TIPP": "IBD/TIPP",
    "Ipsos/Reuters": "Ipsos",
    "NBC/WSJ": "NBC News/Wall Street Journal",
    "Public Policy Polling": "PPP (D)",
    "Pew": "Pew Research Center",
    "Quinnipiac": "Quinnipiac University",
    "Rasmussen": "Rasmussen Reports/Pulse Opinion Research",
    "SurveyMonkey": "SurveyMonkey",
    "YouGov/Economist": "YouGov",
    "Zogby": "Zogby Interactive/JZ Analytics"    
}

In [9]:
# Rename organizations in the Huff Post data frame to match organization names in the 538 data frame.
HuffPost.replace(huff_538_name_dict, inplace=True)
HuffPost.head()

,survey_organization,party_affiliation,start_date,end_date,survey_method,survey_question,survey_population,survey_sample,approve_percent,disapprove_percent,undecided_percent,margin_of_error
0,Gallup,None,2017-01-20,2017-01-22,Live Phone,NaN,Adults,1500.0,45,45,NaN,3.0
1,Ipsos,None,2017-01-20,2017-01-24,Internet,"Overall, do you approve or disapprove about th...",Adults,1633.0,43,45,13.0,2.8
2,Ipsos,None,2017-01-20,2017-01-24,Internet,"Overall, do you approve or disapprove about th...",Adults (Democrats),680.0,15,75,10.0,4.3
3,Ipsos,None,2017-01-20,2017-01-24,Internet,"Overall, do you approve or disapprove about th...",Adults (Republicans),557.0,85,11,4.0,4.7
4,Ipsos,None,2017-01-20,2017-01-24,Internet,"Overall, do you approve or disapprove about th...",Adults (Independents),228.0,38,39,23.0,7.4


In [10]:
# Create a dictionary to expand the abbreviations used by 538 for population to more closely match those used by Huff Post
_538_huff_pop_dict = {
    "a": "Adults",
    "rv": "Registered Voters",
    "v": "Voters",
    "lv": "Likely Voters"
}

In [11]:
# Rename populations in the 538 data frame to more closely match those in the Huff Post data frame
FiveThirtyEight.replace(_538_huff_pop_dict, inplace=True)
FiveThirtyEight['population'].unique()

array(['Adults', 'Registered Voters', 'Likely Voters', 'Voters'],
      dtype=object)

In [12]:
# Create trimmed 538 data frame
_538_trimmed = FiveThirtyEight[["pollster",
                               "sample_size",
                               "population",
                               "methodology",
                               "start_date",
                               "end_date",
                               "yes",
                               "no"
                               ]
                              ]
_538_trimmed.head()

,pollster,sample_size,population,methodology,start_date,end_date,yes,no
0,YouGov,1000,Adults,Online,3/4/20,3/6/20,41.0,51.0
1,YouGov,755,Registered Voters,Online,3/4/20,3/6/20,44.0,55.0
2,YouGov,1000,Adults,Online,3/3/20,3/5/20,43.0,50.0
3,YouGov,763,Registered Voters,Online,3/3/20,3/5/20,45.0,53.0
4,Rasmussen Reports/Pulse Opinion Research,1500,Likely Voters,IVR/Online,3/3/20,3/5/20,47.0,51.0


In [13]:
# Convert start date and end date into datetime objects
_538_trimmed["start_date"] = pd.to_datetime(_538_trimmed["start_date"])
_538_trimmed["end_date"] = pd.to_datetime(_538_trimmed["end_date"])
_538_trimmed.head()

C:\Users\Tristan\anaconda\Anaconda3\envs\pythondata\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Tristan\anaconda\Anaconda3\envs\pythondata\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,pollster,sample_size,population,methodology,start_date,end_date,yes,no
0,YouGov,1000,Adults,Online,2020-03-04,2020-03-06,41.0,51.0
1,YouGov,755,Registered Voters,Online,2020-03-04,2020-03-06,44.0,55.0
2,YouGov,1000,Adults,Online,2020-03-03,2020-03-05,43.0,50.0
3,YouGov,763,Registered Voters,Online,2020-03-03,2020-03-05,45.0,53.0
4,Rasmussen Reports/Pulse Opinion Research,1500,Likely Voters,IVR/Online,2020-03-03,2020-03-05,47.0,51.0


In [14]:
# Create trimmed Huff Post data frame
huff_post_trimmed = HuffPost[["survey_organization",
                              "start_date",
                              "end_date",
                              "survey_method",
                              "survey_population",
                              "survey_sample",
                              "approve_percent",
                              "disapprove_percent",
                              "undecided_percent",
                              "margin_of_error"
                             ]
                            ]
huff_post_trimmed.head()

,survey_organization,start_date,end_date,survey_method,survey_population,survey_sample,approve_percent,disapprove_percent,undecided_percent,margin_of_error
0,Gallup,2017-01-20,2017-01-22,Live Phone,Adults,1500.0,45,45,NaN,3.0
1,Ipsos,2017-01-20,2017-01-24,Internet,Adults,1633.0,43,45,13.0,2.8
2,Ipsos,2017-01-20,2017-01-24,Internet,Adults (Democrats),680.0,15,75,10.0,4.3
3,Ipsos,2017-01-20,2017-01-24,Internet,Adults (Republicans),557.0,85,11,4.0,4.7
4,Ipsos,2017-01-20,2017-01-24,Internet,Adults (Independents),228.0,38,39,23.0,7.4


In [15]:
# Convert start date and end date into datetime objects
huff_post_trimmed["start_date"] = pd.to_datetime(huff_post_trimmed["start_date"], yearfirst=True)
huff_post_trimmed["end_date"] = pd.to_datetime(huff_post_trimmed["end_date"], yearfirst=True)
huff_post_trimmed.head()

C:\Users\Tristan\anaconda\Anaconda3\envs\pythondata\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Tristan\anaconda\Anaconda3\envs\pythondata\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,survey_organization,start_date,end_date,survey_method,survey_population,survey_sample,approve_percent,disapprove_percent,undecided_percent,margin_of_error
0,Gallup,2017-01-20,2017-01-22,Live Phone,Adults,1500.0,45,45,NaN,3.0
1,Ipsos,2017-01-20,2017-01-24,Internet,Adults,1633.0,43,45,13.0,2.8
2,Ipsos,2017-01-20,2017-01-24,Internet,Adults (Democrats),680.0,15,75,10.0,4.3
3,Ipsos,2017-01-20,2017-01-24,Internet,Adults (Republicans),557.0,85,11,4.0,4.7
4,Ipsos,2017-01-20,2017-01-24,Internet,Adults (Independents),228.0,38,39,23.0,7.4


In [16]:
# Create a column name mapper for homogeneity
_538_to_huff_columns = {
    "pollster": "survey_organization",
    "methodology": "survey_method",
    "population": "survey_population",
    "sample_size": "survey_sample",
    "yes": "approve_percent",
    "no": "disapprove_percent"
}

In [17]:
# Rename columns in the trimmed 538 data frame
_538_trimmed.rename(columns=_538_to_huff_columns, inplace=True)
_538_trimmed.head()

C:\Users\Tristan\anaconda\Anaconda3\envs\pythondata\lib\site-packages\pandas\core\frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,survey_organization,survey_sample,survey_population,survey_method,start_date,end_date,approve_percent,disapprove_percent
0,YouGov,1000,Adults,Online,2020-03-04,2020-03-06,41.0,51.0
1,YouGov,755,Registered Voters,Online,2020-03-04,2020-03-06,44.0,55.0
2,YouGov,1000,Adults,Online,2020-03-03,2020-03-05,43.0,50.0
3,YouGov,763,Registered Voters,Online,2020-03-03,2020-03-05,45.0,53.0
4,Rasmussen Reports/Pulse Opinion Research,1500,Likely Voters,IVR/Online,2020-03-03,2020-03-05,47.0,51.0


In [18]:
# Merge the two dataframes
merged_df = _538_trimmed.merge(huff_post_trimmed,
                               how="outer",
                               on=["survey_organization",
                                   "survey_sample",
                                   "survey_population",
                                   "survey_method",
                                   "start_date",
                                   "end_date",
                                   "approve_percent",
                                   "disapprove_percent"
                                  ]
                              )
merged_df.head()

C:\Users\Tristan\anaconda\Anaconda3\envs\pythondata\lib\site-packages\pandas\core\reshape\merge.py:1097: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation
  UserWarning,


,survey_organization,survey_sample,survey_population,survey_method,start_date,end_date,approve_percent,disapprove_percent,undecided_percent,margin_of_error
0,YouGov,1000.0,Adults,Online,2020-03-04,2020-03-06,41.0,51.0,NaN,NaN
1,YouGov,755.0,Registered Voters,Online,2020-03-04,2020-03-06,44.0,55.0,NaN,NaN
2,YouGov,1000.0,Adults,Online,2020-03-03,2020-03-05,43.0,50.0,NaN,NaN
3,YouGov,763.0,Registered Voters,Online,2020-03-03,2020-03-05,45.0,53.0,NaN,NaN
4,Rasmussen Reports/Pulse Opinion Research,1500.0,Likely Voters,IVR/Online,2020-03-03,2020-03-05,47.0,51.0,NaN,NaN


In [19]:
# Homogenize the survey methods
# merged_df["survey_method"].unique()
survey_homogenizer = {
    "Internet": "Online",
    "Live Phone / Online": "Live Phone/Online",
    "Automated Phone/Internet": "Automated Phone/Online"
}
merged_df.replace(survey_homogenizer, inplace=True)

# Check the renaming behaved as expected
merged_df["survey_method"].unique()

array(['Online', 'IVR/Online', 'Automated Phone', 'Live Phone',
       'Live Phone/Online', nan, 'IVR', 'Live PHone',
       'Automated Phone/Online'], dtype=object)

### Create database connection

In [20]:
# create mongo connection
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

db = client.approval

# Remove existing data
db.approval_polls.drop()

Load DataFrames in database:

In [22]:
# insert data into respective collections

db.approval_polls.insert_many(merged_df.to_dict( orient = 'records'))

In [23]:
# query database test
approval_polls = db.approval_polls.find()
for poll in approval_polls:
    print(poll)

{'_id': ObjectId('5e67092800adebab1464fb38'), 'survey_organization': 'YouGov', 'survey_sample': 1000.0, 'survey_population': 'Adults', 'survey_method': 'Online', 'start_date': datetime.datetime(2020, 3, 4, 0, 0), 'end_date': datetime.datetime(2020, 3, 6, 0, 0), 'approve_percent': 41.0, 'disapprove_percent': 51.0, 'undecided_percent': nan, 'margin_of_error': nan}
{'_id': ObjectId('5e67092800adebab1464fb39'), 'survey_organization': 'YouGov', 'survey_sample': 755.0, 'survey_population': 'Registered Voters', 'survey_method': 'Online', 'start_date': datetime.datetime(2020, 3, 4, 0, 0), 'end_date': datetime.datetime(2020, 3, 6, 0, 0), 'approve_percent': 44.0, 'disapprove_percent': 55.0, 'undecided_percent': nan, 'margin_of_error': nan}
{'_id': ObjectId('5e67092800adebab1464fb3a'), 'survey_organization': 'YouGov', 'survey_sample': 1000.0, 'survey_population': 'Adults', 'survey_method': 'Online', 'start_date': datetime.datetime(2020, 3, 3, 0, 0), 'end_date': datetime.datetime(2020, 3, 5, 0, 0)

{'_id': ObjectId('5e67092800adebab14650064'), 'survey_organization': 'Morning Consult', 'survey_sample': 1604.0, 'survey_population': 'Registered Voters', 'survey_method': 'Online', 'start_date': datetime.datetime(2019, 6, 29, 0, 0), 'end_date': datetime.datetime(2019, 7, 1, 0, 0), 'approve_percent': 42.0, 'disapprove_percent': 53.0, 'undecided_percent': nan, 'margin_of_error': nan}
{'_id': ObjectId('5e67092800adebab14650065'), 'survey_organization': 'Morning Consult', 'survey_sample': 1472.0, 'survey_population': 'Registered Voters', 'survey_method': 'Online', 'start_date': datetime.datetime(2019, 6, 29, 0, 0), 'end_date': datetime.datetime(2019, 7, 1, 0, 0), 'approve_percent': 43.0, 'disapprove_percent': 53.0, 'undecided_percent': nan, 'margin_of_error': nan}
{'_id': ObjectId('5e67092800adebab14650066'), 'survey_organization': 'Morning Consult', 'survey_sample': 2200.0, 'survey_population': 'Adults', 'survey_method': 'Online', 'start_date': datetime.datetime(2019, 6, 29, 0, 0), 'end_

{'_id': ObjectId('5e67092800adebab14650834'), 'survey_organization': 'YouGov', 'survey_sample': 1000.0, 'survey_population': 'Adults', 'survey_method': 'Online', 'start_date': datetime.datetime(2018, 9, 22, 0, 0), 'end_date': datetime.datetime(2018, 9, 23, 0, 0), 'approve_percent': 37.0, 'disapprove_percent': 53.0, 'undecided_percent': nan, 'margin_of_error': nan}
{'_id': ObjectId('5e67092800adebab14650835'), 'survey_organization': 'YouGov', 'survey_sample': 859.0, 'survey_population': 'Registered Voters', 'survey_method': 'Online', 'start_date': datetime.datetime(2018, 9, 22, 0, 0), 'end_date': datetime.datetime(2018, 9, 23, 0, 0), 'approve_percent': 42.0, 'disapprove_percent': 55.0, 'undecided_percent': nan, 'margin_of_error': nan}
{'_id': ObjectId('5e67092800adebab14650836'), 'survey_organization': 'Morning Consult', 'survey_sample': 2200.0, 'survey_population': 'Adults', 'survey_method': 'Online', 'start_date': datetime.datetime(2018, 9, 20, 0, 0), 'end_date': datetime.datetime(201

{'_id': ObjectId('5e67092800adebab14651003'), 'survey_organization': 'Gallup', 'survey_sample': 1500.0, 'survey_population': 'Adults', 'survey_method': 'Live Phone', 'start_date': datetime.datetime(2017, 10, 7, 0, 0), 'end_date': datetime.datetime(2017, 10, 9, 0, 0), 'approve_percent': 36.0, 'disapprove_percent': 58.0, 'undecided_percent': nan, 'margin_of_error': nan}
{'_id': ObjectId('5e67092800adebab14651004'), 'survey_organization': 'Rasmussen Reports/Pulse Opinion Research', 'survey_sample': 1500.0, 'survey_population': 'Likely Voters', 'survey_method': 'IVR/Online', 'start_date': datetime.datetime(2017, 10, 5, 0, 0), 'end_date': datetime.datetime(2017, 10, 9, 0, 0), 'approve_percent': 43.0, 'disapprove_percent': 56.0, 'undecided_percent': nan, 'margin_of_error': nan}
{'_id': ObjectId('5e67092800adebab14651005'), 'survey_organization': 'Morning Consult', 'survey_sample': 1996.0, 'survey_population': 'Registered Voters', 'survey_method': 'Online', 'start_date': datetime.datetime(201

In [27]:
# try some experimental queries to verify usability
yougov_polls = db.approval_polls.find({"survey_organization": "YouGov"})
for poll in yougov_polls:
    pprint.pprint(poll)

{'_id': ObjectId('5e67092800adebab1464fb38'),
 'approve_percent': 41.0,
 'disapprove_percent': 51.0,
 'end_date': datetime.datetime(2020, 3, 6, 0, 0),
 'margin_of_error': nan,
 'start_date': datetime.datetime(2020, 3, 4, 0, 0),
 'survey_method': 'Online',
 'survey_organization': 'YouGov',
 'survey_population': 'Adults',
 'survey_sample': 1000.0,
 'undecided_percent': nan}
{'_id': ObjectId('5e67092800adebab1464fb39'),
 'approve_percent': 44.0,
 'disapprove_percent': 55.0,
 'end_date': datetime.datetime(2020, 3, 6, 0, 0),
 'margin_of_error': nan,
 'start_date': datetime.datetime(2020, 3, 4, 0, 0),
 'survey_method': 'Online',
 'survey_organization': 'YouGov',
 'survey_population': 'Registered Voters',
 'survey_sample': 755.0,
 'undecided_percent': nan}
{'_id': ObjectId('5e67092800adebab1464fb3a'),
 'approve_percent': 43.0,
 'disapprove_percent': 50.0,
 'end_date': datetime.datetime(2020, 3, 5, 0, 0),
 'margin_of_error': nan,
 'start_date': datetime.datetime(2020, 3, 3, 0, 0),
 'survey_met

 'survey_sample': 1000.0,
 'undecided_percent': nan}
{'_id': ObjectId('5e67092800adebab1464fc0a'),
 'approve_percent': 45.0,
 'disapprove_percent': 54.0,
 'end_date': datetime.datetime(2020, 1, 24, 0, 0),
 'margin_of_error': nan,
 'start_date': datetime.datetime(2020, 1, 23, 0, 0),
 'survey_method': 'Online',
 'survey_organization': 'YouGov',
 'survey_population': 'Registered Voters',
 'survey_sample': 797.0,
 'undecided_percent': nan}
{'_id': ObjectId('5e67092800adebab1464fc0c'),
 'approve_percent': 43.0,
 'disapprove_percent': 50.0,
 'end_date': datetime.datetime(2020, 1, 24, 0, 0),
 'margin_of_error': nan,
 'start_date': datetime.datetime(2020, 1, 22, 0, 0),
 'survey_method': 'Online',
 'survey_organization': 'YouGov',
 'survey_population': 'Adults',
 'survey_sample': 1000.0,
 'undecided_percent': nan}
{'_id': ObjectId('5e67092800adebab1464fc0d'),
 'approve_percent': 46.0,
 'disapprove_percent': 52.0,
 'end_date': datetime.datetime(2020, 1, 24, 0, 0),
 'margin_of_error': nan,
 'star

 'survey_population': 'Registered Voters',
 'survey_sample': 1117.0,
 'undecided_percent': nan}
{'_id': ObjectId('5e67092800adebab1464fcac'),
 'approve_percent': 39.0,
 'disapprove_percent': 54.0,
 'end_date': datetime.datetime(2019, 12, 19, 0, 0),
 'margin_of_error': nan,
 'start_date': datetime.datetime(2019, 12, 17, 0, 0),
 'survey_method': 'Online',
 'survey_organization': 'YouGov',
 'survey_population': 'Adults',
 'survey_sample': 1000.0,
 'undecided_percent': nan}
{'_id': ObjectId('5e67092800adebab1464fcad'),
 'approve_percent': 44.0,
 'disapprove_percent': 54.0,
 'end_date': datetime.datetime(2019, 12, 19, 0, 0),
 'margin_of_error': nan,
 'start_date': datetime.datetime(2019, 12, 17, 0, 0),
 'survey_method': 'Online',
 'survey_organization': 'YouGov',
 'survey_population': 'Registered Voters',
 'survey_sample': 742.0,
 'undecided_percent': nan}
{'_id': ObjectId('5e67092800adebab1464fcaf'),
 'approve_percent': 42.0,
 'disapprove_percent': 51.0,
 'end_date': datetime.datetime(2019

 'start_date': datetime.datetime(2019, 11, 4, 0, 0),
 'survey_method': 'Online',
 'survey_organization': 'YouGov',
 'survey_population': 'Registered Voters',
 'survey_sample': 731.0,
 'undecided_percent': nan}
{'_id': ObjectId('5e67092800adebab1464fd7d'),
 'approve_percent': 41.0,
 'disapprove_percent': 50.0,
 'end_date': datetime.datetime(2019, 11, 5, 0, 0),
 'margin_of_error': nan,
 'start_date': datetime.datetime(2019, 11, 3, 0, 0),
 'survey_method': 'Online',
 'survey_organization': 'YouGov',
 'survey_population': 'Adults',
 'survey_sample': 1500.0,
 'undecided_percent': nan}
{'_id': ObjectId('5e67092800adebab1464fd7e'),
 'approve_percent': 45.0,
 'disapprove_percent': 53.0,
 'end_date': datetime.datetime(2019, 11, 5, 0, 0),
 'margin_of_error': nan,
 'start_date': datetime.datetime(2019, 11, 3, 0, 0),
 'survey_method': 'Online',
 'survey_organization': 'YouGov',
 'survey_population': 'Registered Voters',
 'survey_sample': 1201.0,
 'undecided_percent': nan}
{'_id': ObjectId('5e67092

 'end_date': datetime.datetime(2019, 9, 27, 0, 0),
 'margin_of_error': nan,
 'start_date': datetime.datetime(2019, 9, 26, 0, 0),
 'survey_method': 'Online',
 'survey_organization': 'YouGov',
 'survey_population': 'Registered Voters',
 'survey_sample': 809.0,
 'undecided_percent': nan}
{'_id': ObjectId('5e67092800adebab1464fe60'),
 'approve_percent': 41.0,
 'disapprove_percent': 53.0,
 'end_date': datetime.datetime(2019, 9, 26, 0, 0),
 'margin_of_error': nan,
 'start_date': datetime.datetime(2019, 9, 25, 0, 0),
 'survey_method': 'Online',
 'survey_organization': 'YouGov',
 'survey_population': 'Adults',
 'survey_sample': 1000.0,
 'undecided_percent': nan}
{'_id': ObjectId('5e67092800adebab1464fe61'),
 'approve_percent': 43.0,
 'disapprove_percent': 55.0,
 'end_date': datetime.datetime(2019, 9, 26, 0, 0),
 'margin_of_error': nan,
 'start_date': datetime.datetime(2019, 9, 25, 0, 0),
 'survey_method': 'Online',
 'survey_organization': 'YouGov',
 'survey_population': 'Registered Voters',
 '

...

 'approve_percent': 41.0,
 'disapprove_percent': 50.0,
 'end_date': datetime.datetime(2018, 9, 18, 0, 0),
 'margin_of_error': nan,
 'start_date': datetime.datetime(2018, 9, 16, 0, 0),
 'survey_method': 'Online',
 'survey_organization': 'YouGov',
 'survey_population': 'Adults',
 'survey_sample': 1500.0,
 'undecided_percent': nan}
{'_id': ObjectId('5e67092800adebab14650865'),
 'approve_percent': 44.0,
 'disapprove_percent': 53.0,
 'end_date': datetime.datetime(2018, 9, 18, 0, 0),
 'margin_of_error': nan,
 'start_date': datetime.datetime(2018, 9, 16, 0, 0),
 'survey_method': 'Online',
 'survey_organization': 'YouGov',
 'survey_population': 'Registered Voters',
 'survey_sample': 1189.0,
 'undecided_percent': nan}
{'_id': ObjectId('5e67092800adebab14650869'),
 'approve_percent': 42.0,
 'disapprove_percent': 51.0,
 'end_date': datetime.datetime(2018, 9, 17, 0, 0),
 'margin_of_error': nan,
 'start_date': datetime.datetime(2018, 9, 16, 0, 0),
 'survey_method': 'Online',
 'survey_organization':

 'survey_sample': 1000.0,
 'undecided_percent': nan}
{'_id': ObjectId('5e67092800adebab146509c7'),
 'approve_percent': 44.0,
 'disapprove_percent': 53.0,
 'end_date': datetime.datetime(2018, 8, 1, 0, 0),
 'margin_of_error': nan,
 'start_date': datetime.datetime(2018, 7, 31, 0, 0),
 'survey_method': 'Online',
 'survey_organization': 'YouGov',
 'survey_population': 'Registered Voters',
 'survey_sample': 866.0,
 'undecided_percent': nan}
{'_id': ObjectId('5e67092800adebab146509ce'),
 'approve_percent': 39.0,
 'disapprove_percent': 51.0,
 'end_date': datetime.datetime(2018, 7, 31, 0, 0),
 'margin_of_error': nan,
 'start_date': datetime.datetime(2018, 7, 29, 0, 0),
 'survey_method': 'Online',
 'survey_organization': 'YouGov',
 'survey_population': 'Adults',
 'survey_sample': 1500.0,
 'undecided_percent': nan}
{'_id': ObjectId('5e67092800adebab146509cf'),
 'approve_percent': 43.0,
 'disapprove_percent': 53.0,
 'end_date': datetime.datetime(2018, 7, 31, 0, 0),
 'margin_of_error': nan,
 'start

 'margin_of_error': nan,
 'start_date': datetime.datetime(2018, 6, 8, 0, 0),
 'survey_method': 'Online',
 'survey_organization': 'YouGov',
 'survey_population': 'Adults',
 'survey_sample': 1000.0,
 'undecided_percent': nan}
{'_id': ObjectId('5e67092800adebab14650b1f'),
 'approve_percent': 40.0,
 'disapprove_percent': 53.0,
 'end_date': datetime.datetime(2018, 6, 8, 0, 0),
 'margin_of_error': nan,
 'start_date': datetime.datetime(2018, 6, 7, 0, 0),
 'survey_method': 'Online',
 'survey_organization': 'YouGov',
 'survey_population': 'Adults',
 'survey_sample': 1000.0,
 'undecided_percent': nan}
{'_id': ObjectId('5e67092800adebab14650b22'),
 'approve_percent': 40.0,
 'disapprove_percent': 52.0,
 'end_date': datetime.datetime(2018, 6, 7, 0, 0),
 'margin_of_error': nan,
 'start_date': datetime.datetime(2018, 6, 6, 0, 0),
 'survey_method': 'Online',
 'survey_organization': 'YouGov',
 'survey_population': 'Adults',
 'survey_sample': 1000.0,
 'undecided_percent': nan}
{'_id': ObjectId('5e670928

 'end_date': datetime.datetime(2017, 10, 31, 0, 0),
 'margin_of_error': nan,
 'start_date': datetime.datetime(2017, 10, 29, 0, 0),
 'survey_method': 'Online',
 'survey_organization': 'YouGov',
 'survey_population': 'Adults',
 'survey_sample': 1500.0,
 'undecided_percent': nan}
{'_id': ObjectId('5e67092800adebab14650f85'),
 'approve_percent': 43.0,
 'disapprove_percent': 52.0,
 'end_date': datetime.datetime(2017, 10, 31, 0, 0),
 'margin_of_error': nan,
 'start_date': datetime.datetime(2017, 10, 29, 0, 0),
 'survey_method': 'Online',
 'survey_organization': 'YouGov',
 'survey_population': 'Registered Voters',
 'survey_sample': 1291.0,
 'undecided_percent': nan}
{'_id': ObjectId('5e67092800adebab14650fac'),
 'approve_percent': 37.0,
 'disapprove_percent': 53.0,
 'end_date': datetime.datetime(2017, 10, 24, 0, 0),
 'margin_of_error': nan,
 'start_date': datetime.datetime(2017, 10, 22, 0, 0),
 'survey_method': 'Online',
 'survey_organization': 'YouGov',
 'survey_population': 'Adults',
 'surv

 'disapprove_percent': 47.0,
 'end_date': datetime.datetime(2017, 2, 14, 0, 0),
 'margin_of_error': 3.1,
 'start_date': datetime.datetime(2017, 2, 12, 0, 0),
 'survey_method': 'Online',
 'survey_organization': 'YouGov',
 'survey_population': 'Adults',
 'survey_sample': 1500.0,
 'undecided_percent': 10.0}
{'_id': ObjectId('5e67092800adebab1465168e'),
 'approve_percent': 14.0,
 'disapprove_percent': 79.0,
 'end_date': datetime.datetime(2017, 2, 14, 0, 0),
 'margin_of_error': nan,
 'start_date': datetime.datetime(2017, 2, 12, 0, 0),
 'survey_method': 'Online',
 'survey_organization': 'YouGov',
 'survey_population': 'Adults (Democrats)',
 'survey_sample': 595.0,
 'undecided_percent': 8.0}
{'_id': ObjectId('5e67092800adebab1465168f'),
 'approve_percent': 86.0,
 'disapprove_percent': 11.0,
 'end_date': datetime.datetime(2017, 2, 14, 0, 0),
 'margin_of_error': nan,
 'start_date': datetime.datetime(2017, 2, 12, 0, 0),
 'survey_method': 'Online',
 'survey_organization': 'YouGov',
 'survey_popul